In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/TSABTAE/sample.csv
/kaggle/input/TSABTAE/test1.csv
/kaggle/input/TSABTAE/train1.csv


# Reference: https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html#sphx-glr-auto-examples-text-plot-document-clustering-py

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
# read data 
train_data= pd.read_csv("/kaggle/input/TSABTAE/train1.csv")#, delimiter ='\t')
test_data= pd.read_csv("/kaggle/input/TSABTAE/test1.csv",)# delimiter ='\t')

Source: https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html#sphx-glr-auto-examples-text-plot-document-clustering-py

In [3]:
from collections import defaultdict
from sklearn import metrics
from time import time

evaluations = []
evaluations_std = []


def fit_and_evaluate(km, X, name=None, n_runs=5):
    name = km.__class__.__name__ if name is None else name

    train_times = []
    scores = defaultdict(list)
    for seed in range(n_runs):
        km.set_params(random_state=seed)
        t0 = time()
        km.fit(X)
        train_times.append(time() - t0)
        scores["Homogeneity"].append(metrics.homogeneity_score(y_train, km.labels_))
        scores["Completeness"].append(metrics.completeness_score(y_train, km.labels_))
        scores["V-measure"].append(metrics.v_measure_score(y_train, km.labels_))
        scores["Adjusted Rand-Index"].append(
            metrics.adjusted_rand_score(y_train, km.labels_)
        )
        scores["Silhouette Coefficient"].append(
            metrics.silhouette_score(X, km.labels_, sample_size=2000)
        )
    train_times = np.asarray(train_times)

    print(f"clustering done in {train_times.mean():.2f} ± {train_times.std():.2f} s ")
    evaluation = {
        "estimator": name,
        "train_time": train_times.mean(),
    }
    evaluation_std = {
        "estimator": name,
        "train_time": train_times.std(),
    }
    for score_name, score_values in scores.items():
        mean_score, std_score = np.mean(score_values), np.std(score_values)
        print(f"{score_name}: {mean_score:.3f} ± {std_score:.3f}")
        evaluation[score_name] = mean_score
        evaluation_std[score_name] = std_score
    evaluations.append(evaluation)
    evaluations_std.append(evaluation_std)

# Feature Extraction using TfidfVectorizer

In [4]:
X=train_data['text']
y=train_data['target']

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.2, random_state=0)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_df=0.8,
    min_df=8,
    stop_words="english",
)
t0 = time()
X_tfidf = vectorizer.fit_transform(np.array(X_train).ravel())

print(f"vectorization done in {time() - t0:.3f} s")
print(f"n_samples: {X_tfidf.shape[0]}, n_features: {X_tfidf.shape[1]}")

vectorization done in 14.168 s
n_samples: 819199, n_features: 31256


In [7]:
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

#data = list(zip(X_train,  y_train))
#kmeans = KMeans(n_clusters=2)
kmeans = KMeans(n_clusters=2, random_state=0, )#n_init="auto")#.fit(X_train)
kmeans.fit(X_tfidf)#  y_train)#data)
kmeans.labels_

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_df=0.8,
    min_df=8,
    stop_words="english",
)
t0 = time()
X_test = vectorizer.fit_transform(np.array(X_test).ravel())

print(f"vectorization done in {time() - t0:.3f} s")
print(f"n_samples: {X_test.shape[0]}, n_features: {X_test.shape[1]}")

vectorization done in 3.481 s
n_samples: 204800, n_features: 12052


In [9]:
identified_clusters = kmeans.fit_predict(X_test)
identified_clusters

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [10]:
from sklearn.metrics import confusion_matrix,classification_report
print("ConfusionMatrix",confusion_matrix(y_test,identified_clusters))
print(classification_report(y_test,identified_clusters))
#ConfusionMatrix

ConfusionMatrix [[96457  5731]
 [90075 12537]]
              precision    recall  f1-score   support

           0       0.52      0.94      0.67    102188
           1       0.69      0.12      0.21    102612

    accuracy                           0.53    204800
   macro avg       0.60      0.53      0.44    204800
weighted avg       0.60      0.53      0.44    204800



# MininBatch

In [11]:
from sklearn.cluster import MiniBatchKMeans
MiniBatchkmeans = MiniBatchKMeans(n_clusters=2,
                          random_state=0,
                          batch_size=10,#100
                          ).fit(X_tfidf)

In [12]:
pred=MiniBatchkmeans.fit_predict(X_test)

In [13]:
from sklearn.metrics import confusion_matrix,classification_report
print("ConfusionMatrix",confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))

ConfusionMatrix [[102178     10]
 [102607      5]]
              precision    recall  f1-score   support

           0       0.50      1.00      0.67    102188
           1       0.33      0.00      0.00    102612

    accuracy                           0.50    204800
   macro avg       0.42      0.50      0.33    204800
weighted avg       0.42      0.50      0.33    204800



# Fuzzy Cmean

In [14]:
pip install fuzzy-c-means

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 391.3 kB/s eta 0:00:00
  Attempting uninstall: tabulate
    Found existing installation: tabulate 0.9.0
    Uninstalling tabulate-0.9.0:
      Successfully uninstalled tabulate-0.9.0
  Attempting uninstall: click
    Found existing installation: click 8.1.3
    Uninstalling click-8.1.3:
      Successfully uninstalled click-8.1.3
  Attempting uninstall: typer
    Found existing installation: typer 0.4.2
    Uninstalling typer-0.4.2:
      Successfully uninstalled typer-0.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 2.2.2 requires click>=8.0, but you have click 7.1.2 which is incompatible.
black 22.6.0 requires click>=8.0.0, but you have click 7.1.2 which is incompatible.
allennlp 2.10.1 requires typer>=0.4.1, but you have typer 0.3.2 which is incompatible.
Note: you may need to restart the ke

import numpy as np
from fcmeans import FCM
my_model = FCM(n_clusters=2) # we use two cluster as an example
my_model.fit(X_tfidf) ## X, numpy array. rows:samples columns:features

centers = my_model.centers
labels = my_model.fit_predict(X_test)

from sklearn.metrics import confusion_matrix,classification_report
print("ConfusionMatrix",confusion_matrix(y_test,labels))
print(classification_report(y_test,labels))

In [15]:
#X_tfidf_array = vectorizer.fit_transform(X_train).toarray()


In [16]:
#X_test_array=vectorizer.fit_transform(X_test).toarray()

In [17]:
#from sklearn.cluster import AgglomerativeClustering
#model = AgglomerativeClustering(n_clusters=2).fit(X_tfidf_array)
# fit model and predict clusters

#yhat = model.fit_predict(X_test)

In [18]:
#yhat = model.fit_predict(X_test_array)

In [19]:
#from sklearn.metrics import confusion_matrix,classification_report
#print("ConfusionMatrix",confusion_matrix(y_test,yhat))
#print(classification_report(y_test,yhat))

1. from sklearn.cluster import Birch
model2 = Birch(threshold=0.01, n_clusters=2)
# fit the model
model2.fit(X_tfidf)
# assign a cluster to each example
yhat2 = model2.fit_predict(X_test)

from sklearn.metrics import confusion_matrix,classification_report
print("ConfusionMatrix",confusion_matrix(y_test,yhat2))
print(classification_report(y_test,yhat2))

# origional test

In [20]:

vectorizer = TfidfVectorizer(
    max_df=0.8,# from 0.5
    min_df=8,# from 5
    stop_words="english",
)
t0 = time()
X_test2 = vectorizer.fit_transform(np.array(test_data['text']).ravel())

print(f"vectorization done in {time() - t0:.3f} s")
print(f"n_samples: {X_test2.shape[0]}, n_features: {X_test2.shape[1]}")

vectorization done in 4.186 s
n_samples: 256000, n_features: 14006


In [21]:
identified_clusters2 = kmeans.fit_predict(X_test2)
identified_clusters2

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [22]:
submission = pd.DataFrame(data = {'id': test_data.id, 'target': identified_clusters2})
submission.to_csv('TFIDFidentified_clusters2(withreval& 0.8 *8 df).csv', index = False)
submission.head()

,id,target
0,1136227,0
1,738556,0
2,1289784,0
3,1419520,0
4,1178091,0


Test MiniBatch

In [23]:
pred2=MiniBatchkmeans.fit_predict(X_test2)

In [24]:
submission = pd.DataFrame(data = {'id': test_data.id, 'target': pred2})
submission.to_csv('TFIDFsubmission_MiniBatch.csv', index = False)
submission.head()

,id,target
0,1136227,0
1,738556,0
2,1289784,0
3,1419520,0
4,1178091,0


from sklearn.cluster import KMeans

for seed in range(5):
    kmeans = KMeans(
        n_clusters=2,
        max_iter=100,
        n_init=1,
        random_state=seed,
    ).fit(X_tfidf)
    cluster_ids, cluster_sizes = np.unique(kmeans.labels_, return_counts=True)
   # print(f"Number of elements asigned to each cluster: {cluster_sizes}")
print()
#print(
  #  "True number of documents in each category according to the class labels: "
 #   f"{category_sizes}"
#)

kmeans = KMeans(
    n_clusters=2,
    max_iter=100,
    n_init=5,
)

fit_and_evaluate(kmeans, X_tfidf, name="KMeans\non tf-idf vectors")

X_tfidf_test = vectorizer.fit_transform(test_data['text'])

print(f"vectorization done in {time() - t0:.3f} s")
print(f"n_samples: {X_tfidf_test.shape[0]}, n_features: {X_tfidf_test.shape[1]}")

# Performing dimensionality reduction using SVD****

In [25]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer


lsa = make_pipeline(TruncatedSVD(n_components=1000), Normalizer(copy=False))
t0 = time()
X_lsa = lsa.fit_transform(X_tfidf)
explained_variance = lsa[0].explained_variance_ratio_.sum()

print(f"LSA done in {time() - t0:.3f} s")
print(f"Explained variance of the SVD step: {explained_variance * 100:.1f}%")

LSA done in 861.090 s
Explained variance of the SVD step: 51.9%


In [26]:
kmeans = KMeans(n_clusters=2, random_state=0, )#n_init="auto")#.fit(X_train)
kmeans.fit(X_lsa)#  y_train)#data)
kmeans.labels_

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [27]:
#lsa = make_pipeline(TruncatedSVD(n_components=1000), Normalizer(copy=False))
#t0 = time()
X_lsa2 = lsa.fit_transform(X_test)
explained_variance = lsa[0].explained_variance_ratio_.sum()

print(f"LSA done in {time() - t0:.3f} s")
print(f"Explained variance of the SVD step: {explained_variance * 100:.1f}%")

LSA done in 1392.353 s
Explained variance of the SVD step: 57.8%


In [28]:
identified_clusters3 = kmeans.fit_predict(X_lsa2)
identified_clusters3

array([1, 1, 1, ..., 1, 1, 0], dtype=int32)

In [29]:
from sklearn.metrics import confusion_matrix,classification_report
print("ConfusionMatrix",confusion_matrix(y_test,identified_clusters3))
print(classification_report(y_test,identified_clusters3))

ConfusionMatrix [[11917 90271]
 [15275 87337]]
              precision    recall  f1-score   support

           0       0.44      0.12      0.18    102188
           1       0.49      0.85      0.62    102612

    accuracy                           0.48    204800
   macro avg       0.46      0.48      0.40    204800
weighted avg       0.47      0.48      0.40    204800



In [30]:
pred5=MiniBatchkmeans.fit_predict(X_lsa2)

In [31]:
from sklearn.metrics import confusion_matrix,classification_report
print("ConfusionMatrix",confusion_matrix(y_test,pred5))
print(classification_report(y_test,pred5))

ConfusionMatrix [[ 8033 94155]
 [ 7684 94928]]
              precision    recall  f1-score   support

           0       0.51      0.08      0.14    102188
           1       0.50      0.93      0.65    102612

    accuracy                           0.50    204800
   macro avg       0.51      0.50      0.39    204800
weighted avg       0.51      0.50      0.39    204800



kmeans = KMeans(
    n_clusters=2,
    max_iter=100,
    n_init=1,
)

fit_and_evaluate(kmeans, X_lsa, name="KMeans\nwith LSA on tf-idf vectors")

In [32]:
#lsa = make_pipeline(TruncatedSVD(n_components=100), Normalizer(copy=False))
#t0 = time()
X_lsa3 = lsa.fit_transform(X_test2)
explained_variance = lsa[0].explained_variance_ratio_.sum()

print(f"LSA done in {time() - t0:.3f} s")
print(f"Explained variance of the SVD step: {explained_variance * 100:.1f}%")

LSA done in 1724.669 s
Explained variance of the SVD step: 56.7%


In [33]:
identified_clusters4 = kmeans.fit_predict(X_lsa3)
identified_clusters4

array([1, 1, 1, ..., 1, 1, 1], dtype=int32)

In [34]:
submission = pd.DataFrame(data = {'id': test_data.id, 'target': identified_clusters4})
submission.to_csv('TFIDFidentified_clusters2&LSA.csv', index = False)
submission.head()

,id,target
0,1136227,1
1,738556,1
2,1289784,1
3,1419520,1
4,1178091,1


In [35]:
pred4=MiniBatchkmeans.fit_predict(X_lsa3)

In [36]:
submission = pd.DataFrame(data = {'id': test_data.id, 'target': pred4})
submission.to_csv('TFIDFsubmission_MiniBatch2&LSA.csv', index = False)
submission.head()

,id,target
0,1136227,1
1,738556,1
2,1289784,1
3,1419520,1
4,1178091,1
